### **Just When I Thought I Was Out, They Pull Me Back In!**

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import io
import os
import base64
import smtplib
import getpass
import zipfile
import openpyxl
import numpy as np
import pandas as pd
from PyPDF2 import PdfReader
from google.colab import files
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

### **"I'll be back..."**

In [ ]:
uploaded = files.upload()

Saving AUG_2024_Invoices_Zipped.zip to AUG_2024_Invoices_Zipped.zip


### **"Life is like a box of chocolates."**

In [ ]:
# Paths to input files
DATA_FILE = 'Testing 1.xlsx'  # Path to the main Excel file containing customer and invoice data

In [ ]:
QR_CODE_IMAGE_PATH = 'QR Code.jpg'  # Path to the QR code image to embed in emails

In [ ]:
# Paths to ZIP files containing attachments
INVOICE_ZIP_FILE_PATH = 'AUG_2024_Invoices_Zipped.zip'  # Path to the ZIP file with PDF invoices

In [ ]:
EXCEL_ZIP_FILE_PATH = 'Aug_24_Excel_Zipped.zip'  # Path to the ZIP file with Excel files
# You can add more ZIP files as needed and define their extraction directories below.

In [ ]:
# Directories to extract ZIP files
INVOICE_EXTRACTED_DIR = '/content/invoices/AUG-2024'  # Directory to extract PDF invoices
EXCEL_EXTRACTED_DIR = '/content/excel_files'  # Directory to extract Excel files
# Add more extraction directories if you have additional ZIP files.

In [ ]:
# Email configuration
SMTP_SERVER = 'smtp.example.com'  # Replace with your SMTP server (e.g., 'smtp.gmail.com')
SMTP_PORT = 587  # Replace with your SMTP port (e.g., 587 for TLS)
SMTP_USER = 'your_email@example.com'  # Replace with your email address
SMTP_PASSWORD = 'your_password'  # Replace with your email password or app-specific password

In [ ]:
FROM_EMAIL = 'your_email@example.com'  # Sender's email address

In [ ]:
# Hardcoded CC and BCC email addresses
CC_EMAILS = ['cc1@example.com', 'cc2@example.com']  # List of CC recipients
BCC_EMAILS = ['bcc1@example.com', 'bcc2@example.com']  # List of BCC recipients

### **"Harry Potter: The Order of the Phoenix" — Helper Functions**

In [ ]:
def ensure_directory(directory_path):
    """
    Ensure that the specified directory exists. If not, create it.
    """
    os.makedirs(directory_path, exist_ok=True)

### **I Feel The Need — The Need for Speed!**

In [ ]:
def extract_zip(zip_file_path, extract_to_dir):
    """
    Extracts the specified ZIP file to the given directory.
    Returns lists of PDF and Excel files found in the extraction.
    """
    print(f"\nExtracting {zip_file_path} to {extract_to_dir}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)
    print(f"Extraction of {zip_file_path} completed.")

    # List all files and subdirectories in the extraction directory
    extracted_items = os.listdir(extract_to_dir)
    print(f"Contents of {extract_to_dir}: {extracted_items}")

    # Initialize lists to hold file paths
    all_files = []

    # Traverse through the extraction directory
    for item in extracted_items:
        item_path = os.path.join(extract_to_dir, item)
        if os.path.isdir(item_path):
            # If item is a directory, list its contents
            sub_files = os.listdir(item_path)
            print(f"Contents of subdirectory {item_path}: {sub_files}")
            for sub_file in sub_files:
                full_sub_file_path = os.path.join(item_path, sub_file)
                all_files.append(full_sub_file_path)
        else:
            all_files.append(item_path)

    # Filter PDF and Excel files
    pdf_files = [f for f in all_files if f.lower().endswith('.pdf')]
    excel_files = [f for f in all_files if f.lower().endswith('.xlsx')]

    print(f"PDF Files Found: {pdf_files}")
    print(f"Excel Files Found: {excel_files}")

    return pdf_files, excel_files

In [ ]:
def check_pdf_contents(pdf_files):
    """
    Checks and prints the contents of each PDF file.
    """
    for pdf_file in pdf_files:
        print(f"\nChecking contents of PDF: {pdf_file}")
        try:
            with open(pdf_file, 'rb') as file:
                reader = PdfReader(file)
                number_of_pages = len(reader.pages)
                print(f"{pdf_file}: {number_of_pages} page(s)")

                if number_of_pages > 0:
                    first_page = reader.pages[0]
                    text = first_page.extract_text()
                    print(f"Text from first page (first 200 characters):\n{text[:200]}...\n")
        except Exception as e:
            print(f"Error reading {pdf_file}: {e}")

def check_excel_contents(excel_files):
    """
    Checks and prints the contents of each Excel file.
    """
    for excel_file in excel_files:
        print(f"\nChecking contents of Excel file: {excel_file}")
        try:
            df_excel = pd.read_excel(excel_file)
            print(f"Columns in {excel_file}: {list(df_excel.columns)}")
            print(f"First few rows of {excel_file}:\n{df_excel.head()}\n")
        except Exception as e:
            print(f"Error reading {excel_file}: {e}")

In [ ]:
def attach_file_to_email(msg, filepath):
    """
    Attaches a file to the email message.
    """
    with open(filepath, 'rb') as file:
        part = MIMEApplication(file.read(), Name=os.path.basename(filepath))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(filepath)}"'
        msg.attach(part)

In [ ]:
def attach_image_to_email(msg, image_path, cid):
    """
    Attaches an image to the email message with a Content-ID.
    """
    with open(image_path, 'rb') as img:
        mime_img = MIMEImage(img.read())
        mime_img.add_header('Content-ID', f'<{cid}>')
        msg.attach(mime_img)

### **Here's Looking at You, Kid**

In [ ]:
def create_email_content(customer_id, customer_records):
    """
    Creates the email subject and body content for a customer.
    """
    subject = f"{customer_id} & INVOICES FOR THE MONTH OF AUG-24"

    # Total Net Amount
    total_net_amt = customer_records['Net_Amt'].sum()

    # Create HTML table for the email body with serial numbers
    table_content = f"""
    <table border="1" cellpadding="4" cellspacing="0" style="border-collapse: collapse; width: 100%;">
        <tr style="background-color: blue; color: white; text-align: center;">
            <th> S.No </th>
            <th>Cust</th>
            <th>Br</th>
            <th>Invoice No</th>
            <th>Invoice Date</th>
            <th>From Date</th>
            <th>To Date</th>
            <th>Invoice Amount</th>
            <th>Net Amount</th>
        </tr>"""

    # Iterating over customer records to generate table rows
    for idx, row in enumerate(customer_records.itertuples(), start=1):
        invoice_date = row.Invoice_Date.strftime('%Y-%m-%d') if not pd.isna(row.Invoice_Date) else ''
        from_date = row.From_Date.strftime('%Y-%m-%d') if not pd.isna(row.From_Date) else ''
        to_date = row.To_Date.strftime('%Y-%m-%d') if not pd.isna(row.To_Date) else ''
        table_content += f"""
        <tr>
            <td style="text-align: center;">{idx}</td>
            <td>{row.Customer_ID}</td>
            <td>{row.Branch}</td>
            <td>{row.Invoice_No}</td>
            <td>{invoice_date}</td>
            <td>{from_date}</td>
            <td>{to_date}</td>
            <td style="text-align: right;">{row.Gross_Amt:.2f}</td>
            <td style="text-align: right;">{row.Net_Amt:.2f}</td>
        </tr>"""

    # Adding the total at the bottom
    table_content += f"""
        <tr>
            <td colspan="7" style="text-align: right;"><b>Net Amount</b></td>
            <td colspan="2" style="text-align: right;"><b>{total_net_amt:.2f}</b></td>
        </tr>
    </table>
    """

    # Email Body
    body_content = f"""
    <p><b style="color:blue; font-size: 20px;">Greetings from Professional Couriers Network Limited!!!</b></p>
    <p>Dear Customer,</p>
    <p>I hope you're well. We are sending you invoices for the month of August-24 with this email, including Total Outstanding as an attachment.</p>

    {table_content}

    <p>We look forward to receiving payment and working with you in the future.</p>
    <br>
    <p>அன்புள்ள வாடிக்கையாளரே, நாங்கள் Aug-24 வரை விலைப்பட்டியல்களை இந்த மின்னஞ்சலுடன் அனுப்புகிறோம், சரிபார்த்து பணம் செலுத்துங்கள்,
    பணம் செலுத்துவதில் ஏதேனும் சிக்கல்கள் அல்லது விலைப்பட்டியல் சிக்கல் இருந்தால் எங்களை தொடர்பு கொள்ளவும்.</p>
    <br>
    <p><b style="color:red;">మేము ఈ ఇమెయిల్‌తో ఆగస్ట్-24 నెల ఇన్‌వాయిస్‌లను పంపుతాము, చెల్లింపు లేదా ఇన్‌వాయిస్‌లలో ఏవైనా సమస్యలు ఉంటే, మేము పరిష్కరించడానికి సిద్ధంగా ఉన్నాము. దయచేసి మమ్మల్ని సంప్రదించండి.</b></p>
    <br>
    <p><b style="color:blue;">ನಾವು ಈ ಇಮೇಲ್ ಮೂಲಕ ಆಗಸ್ಟ್-24 ರ ಇನ್‌ವಾಯಿಸ್‌ಗಳನ್ನು ಕಳುಹಿಸುತ್ತೇವೆ, ಪಾವತಿ ಅಥವಾ ಇನ್‌ವಾಯಿಸ್‌ಗಳಲ್ಲಿ ಯಾವುದೇ ಸಮಸ್ಯೆಗಳಿದ್ದರೆ, ನಾವು ಪರಿಹರಿಸಲು ಸಿದ್ಧರಿದ್ದೇವೆ. ದಯವಿಟ್ಟು ನಮ್ಮನ್ನು ಸಂಪರ್ಕಿಸಿ.</p>
    <br>
    <p><b style="color:green;">ഈ ഇമെയിൽ വഴി ഞങ്ങൾ ഓഗസ്റ്റ്-24-ന് ഇൻവോയ്‌സുകൾ അയയ്‌ക്കുന്നു, പേയ്‌മെൻ്റിലോ ഇൻവോയ്‌സുകളിലോ എന്തെങ്കിലും പ്രശ്‌നങ്ങൾ ഉണ്ടെങ്കിൽ, പരിഹരിക്കാൻ ഞങ്ങൾ തയ്യാറാണ്. ദയവായി ഞങ്ങളെ ബന്ധപ്പെടുക.</p>
    <br>
    <p><b>Beneficiary Name: Professional Couriers Network Ltd</b><br>
    Bank Name: HDFC Bank<br>
    Bank Account Number: 50200053051782<br>
    IFSC Code: HDFC0002461</p>
    <br>
    <img src="cid:qr_code_image" alt="QR Code" width="250" height="250">
    <br>
    <p>Yours sincerely,<br>
    Ameer Sultan<br>
    CAO / PCNL<br>
    Al Barakkah Towers<br>
    10. K.B. Dasan Road, Alwarpet,<br>
    Chennai - 600 018<br>
    Mobile: +91 965 001 8307<br>
    Email: ameersultan@pcnl.in<br>
    website: www.tpcindia.com</p>
    """

    return subject, body_content

### **You Had Me at Hello**

In [ ]:
def send_email_with_attachments(to_email, subject, body, pdf_filepaths, excel_filepaths, additional_filepaths=[]):
    """
    Constructs and sends an email with the specified attachments, CC, and BCC.
    Supports multiple PDF, Excel, and additional file attachments.
    """
    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = FROM_EMAIL
    msg['To'] = to_email
    msg['Cc'] = ', '.join(CC_EMAILS)  # Include CC emails in the email header
    msg['Subject'] = subject

    # Attach the email body
    msg.attach(MIMEText(body, 'html'))

    # Attach the QR code image
    if os.path.exists(QR_CODE_IMAGE_PATH):
        attach_image_to_email(msg, QR_CODE_IMAGE_PATH, 'qr_code_image')
    else:
        print(f"QR code image not found at {QR_CODE_IMAGE_PATH}. Skipping attachment.")

    # Attach PDF Invoices
    for pdf_filepath in pdf_filepaths:
        if os.path.exists(pdf_filepath):
            attach_file_to_email(msg, pdf_filepath)
        else:
            print(f"PDF file not found at {pdf_filepath}. Skipping attachment.")

    # Attach Excel Files
    for excel_filepath in excel_filepaths:
        if os.path.exists(excel_filepath):
            attach_file_to_email(msg, excel_filepath)
        else:
            print(f"Excel file not found at {excel_filepath}. Skipping attachment.")

    # Attach Additional Files
    for additional_filepath in additional_filepaths:
        if os.path.exists(additional_filepath):
            attach_file_to_email(msg, additional_filepath)
        else:
            print(f"Additional file not found at {additional_filepath}. Skipping attachment.")

    # Combine all recipients for sending (To, CC, BCC)
    recipients = [to_email] + CC_EMAILS + BCC_EMAILS

    # Send the email via SMTP
    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls()  # Secure the connection
            smtp.login(SMTP_USER, SMTP_PASSWORD)  # Log in to the SMTP server
            smtp.sendmail(FROM_EMAIL, recipients, msg.as_string())  # Send the email
        print(f"Email successfully sent to {to_email} (CC: {', '.join(CC_EMAILS)}; BCC: {', '.join(BCC_EMAILS)})")
    except Exception as e:
        print(f"Failed to send email to {to_email}: {e}")

### **You Can't Handle the Truth!**

In [ ]:
def main():
    # Ensure extraction directories exist
    ensure_directory(INVOICE_EXTRACTED_DIR)
    ensure_directory(EXCEL_EXTRACTED_DIR)
    # Add more directories here if you have additional ZIP files

    # Step 1: Extract Invoices ZIP file
    invoice_pdfs, _ = extract_zip(INVOICE_ZIP_FILE_PATH, INVOICE_EXTRACTED_DIR)

    # Step 2: Extract Excel ZIP file
    _, excel_files = extract_zip(EXCEL_ZIP_FILE_PATH, EXCEL_EXTRACTED_DIR)

    # (Optional) Check contents of extracted files
    check_pdf_contents(invoice_pdfs)
    check_excel_contents(excel_files)

    # Create mappings for invoices and Excel files
    # Mapping invoice_no to PDF file path
    invoice_dict = {}
    for pdf_file in invoice_pdfs:
        base_name = os.path.splitext(os.path.basename(pdf_file))[0]  # e.g., "Invoice_JPRTR100206"
        if base_name.startswith('Invoice_'):
            invoice_no = base_name.replace('Invoice_', '')
            invoice_dict[invoice_no] = pdf_file

    # Mapping customer_id to Excel file path
    excel_dict = {}
    for excel_file in excel_files:
        customer_id = os.path.splitext(os.path.basename(excel_file))[0]  # e.g., "AKV"
        excel_dict[customer_id] = excel_file

    # (Optional) Print mappings for debugging
    print("\nInvoice Mapping:")
    for k, v in invoice_dict.items():
        print(f"{k}: {v}")

    print("\nExcel Mapping:")
    for k, v in excel_dict.items():
        print(f"{k}: {v}")

    # Loop through each unique customer in the DataFrame
    for customer_id in df['Customer_ID'].unique():
        # Filter records for the current customer
        customer_records = df[df['Customer_ID'] == customer_id]

        # Retrieve the primary email address
        to_email = customer_records.iloc[0]['To_Mail_ID']  # Get 'To_Mail_ID' from the first row

        # Create email subject and body
        subject, body_content = create_email_content(customer_id, customer_records)

        # Define file paths based on the first record
        invoice_no = customer_records.iloc[0]['Invoice_No']
        pdf_filepath = invoice_dict.get(invoice_no, None)

        excel_filepath = excel_dict.get(customer_id, None)

        # Debugging: Print the paths being used
        print(f"\nPreparing to send email to {to_email} for Customer ID {customer_id}")
        print(f"PDF File Path: {pdf_filepath}")
        print(f"Excel File Path: {excel_filepath}")

        # If you have additional attachment directories or files, gather them here
        # For example, attaching all files in an 'additional_files' directory related to the customer
        # additional_files_dir = '/content/additional_files'
        # additional_filepaths = [os.path.join(additional_files_dir, f) for f in os.listdir(additional_files_dir) if customer_id in f]
        # send_email_with_attachments(to_email, subject, body_content, [pdf_filepath], [excel_filepath], additional_filepaths)

        # For now, we'll assume no additional attachments
        send_email_with_attachments(to_email, subject, body_content, [pdf_filepath] if pdf_filepath else [], [excel_filepath] if excel_filepath else [])


### **Houston, We Have a Problem**

In [ ]:
if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.

   Balance  
0     1276  
1      691  
2      137  
3      684  
4     1402  


Checking contents of Excel file: /content/excel_files/aug-2024/aug-24/SBD.xlsx
Columns in /content/excel_files/aug-2024/aug-24/SBD.xlsx: ['Customer', 'CompCode', 'Customer/Franchisee', 'Inv.No', 'Inv.Date', 'Inv.Value', 'Balance']
First few rows of /content/excel_files/aug-2024/aug-24/SBD.xlsx:
  Customer CompCode Customer/Franchisee       Inv.No   Inv.Date Inv.Value  \
0      SBD      DEL         TPCSHAHABAD  DELTR006585 2021-03-05      4224   
1      SBD      DEL         TPCSHAHABAD  DELTR006728 2021-04-05     64121   
2      SBD      DEL         TPCSHAHABAD  DELTR006931 2021-05-05     48061   
3      NaN      NaN                 NaN          NaN        NaT     Total   

   Balance  
0     4224  
1    64121  
2    48061  
3   116406  


Checking contents of Excel file: /content/excel_files/aug-2024/aug-24/KHH.xlsx
Columns in /content/excel_files/aug-2024

#---------------------------------------- **The End**--------------------------------------------------